In [0]:
spark.conf.set("fs.azure.account.key.adlsbikestoreinterns.blob.core.windows.net", "2A7Fbw9UkJ7R03r/x+uw6/JNClb4b73dpL+KGsIVfAOpJfRVBjH1rjMC2xLTj4uVCR/j8qgXfabp+AStWjlnNg==")
dbutils.secrets.listScopes()
dbutils.secrets.list(scope = "bikes-scope")
bikes_account_key = dbutils.secrets.get(scope = "bikes-scope", key = "account-key")
spark.conf.set(
    "fs.azure.account.key.adlsbikestoreinterns.blob.core.windows.net",
    bikes_account_key
)

In [0]:
%run ./utilities

In [0]:
load_source_data(spark, metadata)

{'brands': DataFrame[brand_id: string, brand_name: string, time_stamp: string, process_id: string],
 'order_items': DataFrame[order_id: string, item_id: string, product_id: string, quantity: string, list_price: string, discount: string, time_stamp: string, process_id: string],
 'staffs': DataFrame[staff_id: string, first_name: string, last_name: string, email: string, phone: string, active: string, store_id: string, manager_id: string, time_stamp: string, process_id: string],
 'categories': DataFrame[category_id: string, category_name: string, time_stamp: string, process_id: string],
 'customers': DataFrame[customer_id: string, first_name: string, last_name: string, phone: string, email: string, street: string, city: string, state: string, zip_code: string, time_stamp: string, process_id: string],
 'order': DataFrame[order_id: int, customer_id: int, order_status: int, order_date: string, required_date: string, shipped_date: string, store_id: int, staff_id: int, time_stamp: string, proc

In [0]:
dataframes = {}
for df_name, df in [(df_name, globals()[df_name]) for df_name in dir() if df_name.endswith("_df")]:
    dataframes[df_name[:-3]] = df
display(order_items_df)

In [0]:
categories_df =handle_duplicates(categories_df,metadata=metadata,table_name='categories')
categories_df = handle_nulls(categories_df,metadata=metadata,table_name='categories')

In [0]:
customers_df =check_column_data_type(customers_df,'customers',metadata)

In [0]:
customers_df=standardize_name_case(customers_df,'first_name')
customers_df=standardize_name_case(customers_df,'last_name')
customers_df = handle_duplicates(customers_df, metadata, table_name ='customers')
customers_df = handle_nulls(customers_df,metadata=metadata,table_name='customers')
customers_df=handle_outliers(customers_df, metadata['table_name'],method='zscore')
customers_df =handle_outliers(customers_df,metadata['table_name'], method='iqr')
customers_df =check_column_data_type(customers_df,'customers',metadata)
display(customers_df)

In [0]:
#staffs_df = handle_duplicates(staffs_df, metadata, table_name ='staffs')
#staffs_df=handle_nulls(staffs_df,metadata=metadata,table_name='staffs')

In [0]:
order_df = handle_duplicates(order_df, metadata, table_name ='order')
order_df = handle_nulls(order_df,metadata=metadata,table_name='order')

In [0]:
#order_items_df = handle_duplicates(order_items_df, metadata, table_name ='order_items')
#order_items_df = handle_duplicates(order_items_df, metadata, table_name ='order_items')
order_items_df = handle_outliers(order_items_df, column = 'list_price' , method='zscore')
order_items_df = handle_outliers(order_items_df, column = 'quantity' , method='zscore')
order_items_df = handle_outliers(order_items_df, column = 'discount' , method='zscore')
display(order_items_df)

In [0]:
brands_df =handle_duplicates(brands_df, metadata, table_name ='brands')
brands_df=handle_nulls(brands_df,metadata=metadata,table_name='brands')

In [0]:
products_df = handle_duplicates(products_df,metadata=metadata, table_name='products')
products_df = handle_nulls(products_df,metadata=metadata,table_name='products')
products_df = handle_outliers(products_df, column = 'model_year' , method='zscore')
products_df = handle_outliers(products_df, column = 'list_price' , method='iqr')
products_df = check_column_data_type(products_df, table_name='products_id', metadata_df=metadata)
display(products_df)

In [0]:
stores_df = handle_duplicates(stores_df ,metadata = metadata, table_name='stores' )
stores_df = handle_nulls(stores_df ,metadata = metadata, table_name='stores' )
display(stores_df)

store_id,store_name,phone,email,street,city,state,zip_code,time_stamp,process_id
4,Mohamed Ali,NULL,Mohamed.d.ali@pwc.com,101 Good Morning,Cairo,NULL,11835,2024-08-29T19:54:49.7832203Z,914fbcbb-bfb6-48fd-a12f-48b89a59f9e2
2,Baldwin Bikes,(516) 379-8888,baldwin@bikes.shop,4200 Chestnut Lane,Baldwin,NY,11432,2024-08-29T19:54:49.7832203Z,914fbcbb-bfb6-48fd-a12f-48b89a59f9e2
3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088,2024-08-29T19:54:49.7832203Z,914fbcbb-bfb6-48fd-a12f-48b89a59f9e2
1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060,2024-08-29T19:54:49.7832203Z,914fbcbb-bfb6-48fd-a12f-48b89a59f9e2


In [0]:
for table_name, df in dataframes.items():
    # Write the cleaned DataFrame to a Delta table in ADF storage account
    df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"wasbs://dlbikestorestage1@adlsbikestoreinterns.blob.core.windows.net/mostafa/{table_name}/")